# Imports

In [ ]:
library(R.matlab)
library(vars)
library("matrixStats")
set.seed(18)
options(warn=-1) 

In [2]:
# download landslide data
mine = readMat('data/mine_data.mat')
mine = mine$data

In [3]:
find_distance = function(x,y){
  return(sqrt((x[1]-y[1])^2+(x[2]-y[2])^2))
}
n.index = c(465,759,769,259,396,1444,90,175,281,507,1662,1038,429,821,1315,63,941,999,929,800,1014,1174,394,1701,825,184,994,480,1555,41)


# Non-stationary

In [29]:
generate_w = function(n.index,alpha, d_cutoff){
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
        W[i,j] = alpha*(-log(find_distance(mine[i,2:3],mine[j,2:3])+1)+log(d_cutoff))
#         W[i,j] = exp(W[i,j])
        
    }
    
#     W[i,] = W[i,] / sum(W[i,])
    
  }
  return(W)
}

# function on generate simulated data at one step given previous value, spatial weight matrix, mean function and noise
generate_one_step = function(x.t_1, W.t_1,epsilon.t_t){
  n = dim(W.t_1)[1]
  x.t = matrix(0,nrow = n,ncol = 1)
  x.t = W.t_1 %*% x.t_1  + epsilon.t_t  # this is the form of our non-stationary model
  return(x.t)
}

# The function aims to simulate all data. It iteratively apply generate_one_step function
generate_sim = function(x_1,alpha.bound,T,epsilon,d_cutoff){
  x = cbind(x_1)
  n = dim(x_1)[1]
  for(t in 2:T){
    alpha.t_1 = (1 - (t-1)/T)*alpha.bound[1]+(t-1)/T*alpha.bound[2]
    W.t_1 = generate_w(n.index,alpha.t_1,d_cutoff)
    tem = matrix(0,nrow = n,ncol = 1)
    tem = generate_one_step(matrix(x[,ncol(x)],nrow = n),W.t_1,epsilon[,t])
    x = cbind(x,tem)
  }
  return(x)
}

In [34]:
path = 'data/non_stationary/csv/'
n = 100
alpha.m = matrix(0, nrow=n, ncol=2)

In [35]:
for (i in 0:(n-1)) {
    file_ = paste0('s',as.character(i))
    file_ = paste0(file_,'.csv',seq = "")
    file_name = file.path(path,file_)

    # create epsilon_t
    epsilon_t = matrix(rnorm(30*500,0,1),nrow = 30)
    # initial x_1 at time=1
    initial_value = matrix(runif(30,-0.1,0.1),nrow = 30,ncol = 1)
    alpha.bound = runif(1,1,2)*(10^-4)
    d_cutoff = 170
    x = generate_sim(initial_value,c(alpha.bound,10*alpha.bound),500,epsilon_t,d_cutoff)

    alpha.m[i+1, ] = c(alpha.bound,10*alpha.bound)

    write.csv(x,file_name)
}

In [36]:
write.csv(alpha.m, 'data/non_stationary/csv/alphas.csv')

In [ ]:
X = read.csv('data/non_stationary/csv/s2.csv')
X = as.matrix(X[, 2:501])
plot(X[1, ])

# Stationary

In [4]:
# W[i,j] = alpha * find_distance(mine[i,2:3],mine[j,2:3])^2 -400*alpha* find_distance(mine[i,2:3],mine[j,2:3]) + 40000*alpha

generate_w = function(n.index,alpha,d_cutoff){
  W = matrix(0,nrow = length(n.index),ncol = length(n.index))
  for (i in 1:length(n.index)){
    for (j in 1:length(n.index)) {
        W[i,j] = alpha*(-log(find_distance(mine[i,2:3],mine[j,2:3])+1)+log(d_cutoff))
#         W[i,j] = exp(W[i,j])

      
    }
#       W[i,] = W[i,] / sum(W[i,])

  }
  return(W)
}

In [24]:
path = 'data/stationary/csv/'

In [25]:
n = 100
alpha.m = matrix(0, nrow=n, ncol=1)

In [26]:
for (i in 0:(n-1)) {
    file_ = paste0('s',as.character(i))
    file_ = paste0(file_,'.csv',seq = "")
    file_name = file.path(path,file_)
    x1 = runif(30,-0.01,0.01) # 
    
    X = matrix(0,nrow = 30,ncol = 500)
    X[,1] = x1

    alpha = runif(1,0.05,0.06) # log
#     alpha = runif(1,0.0000001,0.0000009) # quadratic
    d_cutoff = 170

    W = generate_w(n.index,alpha,d_cutoff)
    epsilon_t = matrix(rnorm(30*500,0,1),nrow = 30)

    for (t in 2:500){
        X[,t] = W %*% (matrix(X[,t-1],ncol = 1))+epsilon_t[,t]
      }

    write.csv(X,file_name)
    alpha.m[i+1, ] = alpha
    write.csv(alpha.m, paste0(path, 'alphas.csv'))
}

In [ ]:
X = read.csv('data/stationary/csv/s2.csv')
X = as.matrix(X[, 2:501])
plot(X[1, ])